In [1]:
# load library
library(dplyr)


Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



In [2]:
# laod the soccer database
soccerDB <- src_sqlite("database.sqlite")


In [3]:
# load all required tables into separate dataframes
matchTable <- collect(tbl(soccerDB,"Match"))
playerTable <- collect(tbl(soccerDB,"Player"))
player_AttributesTable <- collect(tbl(soccerDB,"Player_Attributes"), n = Inf)
teamTable <- collect(tbl(soccerDB,"Team"))
team_AttributesTable <- collect(tbl(soccerDB,"Team_Attributes"))


In [4]:
# inner join player attirbutes table with player table on player_api_id 
# to get all information on players in one dataframe
player_AttributesTable <- inner_join(player_AttributesTable, playerTable, by="player_api_id") %>% 
                            group_by(player_api_id) %>% 
                            filter(date == max(date))


In [5]:
# create gk_rating for all players by adding all the gk columns
# [gk_handling, gk_kicking, gk_reflexes, gk_positioning]
# keep only required columns to reduce dataframe size 
# [player_api_id, name, overall_rating, gk_rating]
player_AttributesTable <- player_AttributesTable %>% 
    mutate(gk_rating = (gk_handling + gk_kicking + gk_reflexes + gk_positioning)) %>% 
    select(player_api_id, player_name, overall_rating, gk_rating)


In [6]:
# create 22 different variables to store inner_join between player_attribute and matchTable 
# match by checking player id with the different player columns [home_player_1, away_player_1, etc.]
# group all of these joins by player_api_id and league id to get the count of matches 
# played by any player in each league
for(team in c('home','away')){
    for(num in 1:11){
        colName <- paste(team, "player", num, sep="_")
        nam <- paste(colName)
        assign(nam, inner_join(player_AttributesTable, matchTable, by = c("player_api_id" = colName)) %>% 
               group_by(player_api_id, league_id) %>% summarize(num_appearences = n())
               )
    }
}         

In [7]:
# combine all the 22 variables to get one dataframe
ResultDF <- rbind(home_player_1, away_player_1)
ResultDF <- rbind(ResultDF, home_player_2)
ResultDF <- rbind(ResultDF, home_player_3)
ResultDF <- rbind(ResultDF, home_player_4)
ResultDF <- rbind(ResultDF, home_player_5)
ResultDF <- rbind(ResultDF, home_player_6)
ResultDF <- rbind(ResultDF, home_player_7)
ResultDF <- rbind(ResultDF, home_player_8)
ResultDF <- rbind(ResultDF, home_player_9)
ResultDF <- rbind(ResultDF, home_player_10)
ResultDF <- rbind(ResultDF, home_player_11)
ResultDF <- rbind(ResultDF, away_player_2)
ResultDF <- rbind(ResultDF, away_player_3)
ResultDF <- rbind(ResultDF, away_player_4)
ResultDF <- rbind(ResultDF, away_player_5)
ResultDF <- rbind(ResultDF, away_player_6)
ResultDF <- rbind(ResultDF, away_player_7)
ResultDF <- rbind(ResultDF, away_player_8)
ResultDF <- rbind(ResultDF, away_player_9)
ResultDF <- rbind(ResultDF, away_player_10)
ResultDF <- rbind(ResultDF, away_player_11)

In [8]:
# group the result dataframe by player_api_id to get count of matches [sum]
# and count of distinct leagues [n_distinct]
# join this dataframe with player attributes to get all information in one dataframe
player_AttributesTable <- ResultDF %>% 
                            group_by(player_api_id) %>% 
                            summarize(num_appearences = sum(num_appearences),
                                     num_leagues = n_distinct(league_id)) %>% 
                            inner_join(player_AttributesTable, by = "player_api_id")

In [9]:
# preview of the player attributes database
arrange(player_AttributesTable, desc(num_leagues))

player_api_id,num_appearences,num_leagues,player_name,overall_rating,gk_rating
42119,175,6,Gelson Fernandes,76,42
116750,151,5,Ricky van Wolfswinkel,73,46
178538,105,5,Filip Djuricic,75,40
22543,147,4,Radamel Falcao,82,33
24123,130,4,Jonathan De Guzman,76,36
25816,68,4,Almen Abdi,72,33
26434,94,4,Ibrahim Afellay,78,37
26489,89,4,Edson Braafheid,72,39
26675,141,4,Eljero Elia,75,42
30477,63,4,Danijel Pranjic,73,41


In [10]:
# remove all rows where overall_rating or gk_rating is NA
player_AttributesTable <- filter(player_AttributesTable, !is.na(overall_rating), !is.na(gk_rating))

In [11]:
# write the final dataframe to csv
write.csv(player_AttributesTable, file = "players.csv", row.names = FALSE)

In [12]:
# convert all NAs to 0 in team attributes
# join team attributes with team to get all information on teams in one dataframe
# create buildUpPlay from all buildUp attributes columns
# [buildUpPlayDribbling, buildUpPlayPassing, buildUpPlaySpeed]
# create chanceCreation from all chance attributes columns
# [chanceCreationCrossing, chanceCreationPassing, chanceCreationShooting]
# create defence from all defence attributes columns
# [defenceAggression, defencePressur, defenceTeamWidth]
# select only required columns [team_api_id, name, buildUpPlay, chanceCreation, defence]
team_AttributesTable <- team_AttributesTable %>% mutate_all(funs(replace(., is.na(.), 0)))
team_AttributesTable <- teamTable %>% 
                        inner_join(team_AttributesTable, by = "team_api_id") %>%
                        group_by(team_api_id) %>%
                        filter(date == max(date)) %>%
                        mutate(buildUpPlay = (buildUpPlayDribbling + buildUpPlayPassing + buildUpPlaySpeed),
                              chanceCreation = (chanceCreationCrossing + chanceCreationPassing + chanceCreationShooting),
                              defence = (defenceAggression + defencePressure + defenceTeamWidth)) %>%
                        select(team_api_id, team_long_name, buildUpPlay, chanceCreation, defence)

In [13]:
# preview of the team attributes dataframe
team_AttributesTable

team_api_id,team_long_name,buildUpPlay,chanceCreation,defence
9987,KRC Genk,148,155,163
9993,Beerschot AC,86,151,158
10000,SV Zulte-Waregem,147,131,139
9994,Sporting Lokeren,159,154,149
9984,KSV Cercle Brugge,145,152,168
8635,RSC Anderlecht,149,160,164
9991,KAA Gent,142,142,155
9998,RAEC Mons,160,168,138
9985,Standard de Liège,161,173,151
8203,KV Mechelen,134,150,145


In [14]:
# create 22 different variables to store inner_join between team_attribute and matchTable 
# match by checking team api id with the different team columns [home_team_api_id, away_plaway_team_api_id]
# group by team api id
# select useful columns to reduce dataframe size [team id, team name, player id]
# group all of these joins by team_api_id, team_long_name and player id to get all unique players for each team
matchResultDF <- data.frame()
for(team in c('home','away')){
    for(num in 1:11){
        colName <- paste(team, "player", num, sep="_")
        joinColName <- paste(team, "team_api_id", sep="_")
        nam <- paste(colName)
        assign(nam, inner_join(team_AttributesTable, matchTable, by = c("team_api_id" = joinColName)) %>% 
                        group_by(team_api_id) %>% 
                        select_(.dots = c("team_api_id","team_long_name","player_id" = colName)) %>% 
                        filter(!is.na(player_id)) %>%
                        group_by(team_api_id, team_long_name, player_id) %>% 
                        summarize(count = 1) %>% 
                        select(1: 3) 
                )
    }
}


In [15]:
# combine all the variables into one dataframe
matchResultDF <- rbind(home_player_1, away_player_1)
matchResultDF <- rbind(matchResultDF, home_player_2)
matchResultDF <- rbind(matchResultDF, home_player_3)
matchResultDF <- rbind(matchResultDF, home_player_4)
matchResultDF <- rbind(matchResultDF, home_player_5)
matchResultDF <- rbind(matchResultDF, home_player_6)
matchResultDF <- rbind(matchResultDF, home_player_7)
matchResultDF <- rbind(matchResultDF, home_player_8)
matchResultDF <- rbind(matchResultDF, home_player_9)
matchResultDF <- rbind(matchResultDF, home_player_10)
matchResultDF <- rbind(matchResultDF, home_player_11)
matchResultDF <- rbind(matchResultDF, away_player_2)
matchResultDF <- rbind(matchResultDF, away_player_3)
matchResultDF <- rbind(matchResultDF, away_player_4)
matchResultDF <- rbind(matchResultDF, away_player_5)
matchResultDF <- rbind(matchResultDF, away_player_6)
matchResultDF <- rbind(matchResultDF, away_player_7)
matchResultDF <- rbind(matchResultDF, away_player_8)
matchResultDF <- rbind(matchResultDF, away_player_9)
matchResultDF <- rbind(matchResultDF, away_player_10)
matchResultDF <- rbind(matchResultDF, away_player_11)



In [16]:
# group the dataframe by team id to get all the players who have played for them [n_distinct]
# join the resulting dataframe with team attributes table to get all information in one dataframe
team_AttributesTable <- matchResultDF %>% 
    group_by(team_api_id) %>% 
    summarize(num_fielded_players = n_distinct(player_id)) %>% 
    inner_join(team_AttributesTable, by = "team_api_id")

In [17]:
# preview of the final dataframe
team_AttributesTable

team_api_id,num_fielded_players,team_long_name,buildUpPlay,chanceCreation,defence
1601,55,Ruch Chorzów,133,176,136
1773,55,Oud-Heverlee Leuven,153,157,137
1957,64,Jagiellonia Białystok,158,152,162
2033,87,S.C. Olhanense,147,134,105
2182,60,Lech Poznań,147,196,153
2183,35,P. Warszawa,79,131,139
2186,49,Cracovia,125,165,157
4087,56,Évian Thonon Gaillard FC,149,164,140
4170,31,US Boulogne Cote D'Opale,86,154,155
6269,28,Novara,179,168,138


In [18]:
# write the final teams dataframe to csv
write.csv(team_AttributesTable, file = "teams.csv", row.names = FALSE)